## Preparation

In [1]:
%pip install nltk

/bin/bash: /anaconda/envs/jupyter_env/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Note: you may need to restart the kernel to use updated packages.


In [2]:
# General imports

# NLP imports
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize

# Azure imports
from azureml.core import Workspace, Dataset

Prepare NLTK

In [3]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/azureuser/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/azureuser/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Import dataset from database

In [4]:
subscription_id = '<<subscription_id>>'
resource_group = 'rg-leore-001'
workspace_name = 'leomathesisML'

workspace = Workspace(subscription_id, resource_group, workspace_name)

dataset = Dataset.get_by_name(workspace, name='db_data_asset')
df = dataset.to_pandas_dataframe()

Failed to extract subscription information, Exception=AttributeError; 'Logger' object has no attribute 'activity_info'
Failed to extract subscription information, Exception=AttributeError; 'Logger' object has no attribute 'activity_info'
Failed to extract subscription information, Exception=AttributeError; 'Logger' object has no attribute 'activity_info'
Failed to extract subscription information, Exception=AttributeError; 'Logger' object has no attribute 'activity_info'
Failed to extract subscription information, Exception=AttributeError; 'Logger' object has no attribute 'activity_info'
Message: rslex failed, falling back to clex.
Payload: {"pid": 44967, "source": "azureml.dataprep", "version": "4.8.4", "trace": "azureml|data|tabular_dataset.py, line 169 in function <lambda>.\nazureml|data|dataset_error_handling.py, line 106 in function _try_execute.\nazureml|data|tabular_dataset.py, line 169 in function to_pandas_dataframe.", "subscription": "", "run_id": "", "resource_group": "", "w

## Filter dataset

Temporary filter by date

In [5]:
df = df[(df['date'].dt.year >= 2010)]

General filters

In [6]:
df = df[df['text'].isnull() == False]
df = df[df['text'].str.split().apply(len) >= 25]
df['text'] = [t.replace("\n", " ").replace("\r", " ").strip() for t in df['text']]
len(df)

59246

## Generate word- and sentence-tokens

In [7]:
df['sentences'] = df['text'].apply(sent_tokenize)
df['tokens'] = df['text'].apply(word_tokenize)

## Create sliding window over sentences

In [8]:
def create_sliding_windows(i_list, window_size=4):
  return_list = []
  if len(i_list) <= window_size:
    return_list.append(" ".join(i_list))
  else:
    for i in range(len(i_list) - window_size + 1):
      return_list.append(" ".join(i_list[i:i + window_size]))
  return return_list

In [9]:
df['sentences_window'] = df['sentences'].apply(create_sliding_windows)
df.drop('sentences', axis=1, inplace=True)

In [10]:
df = df.explode('sentences_window', ignore_index=True)

In [11]:
df.reset_index(inplace=True)
df = df.rename(columns = {'index':'window_id'})

In [12]:
len(df)

674069

In [13]:
df.to_pickle("../data/df_ml_ready_2010.pkl")